In [7]:
import pycarrot.nn as nn
import pycarrot.optim as optim
from pycarrot.carrot import Carrot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pycarrot
import pycarrot.nn.functional as F

In [8]:
# build data class, mainly get_iem
class MnistDataset(object):

    def __init__(self, csv_file):
        self.df_data = pd.read_csv(csv_file, header=None)
        pass

    def __len__(self):
        return len(self.df_data)

    def __getitem__(self, index):
        image_label = self.df_data.iloc[index, 0]
        target = Carrot(np.zeros((10)))
        target.data[image_label] = 1.0

        image_content = np.array(self.df_data.iloc[index, 1:].values) / 255.0
        image_content = Carrot(image_content)

        return image_label, image_content, target
    pass

In [9]:
# build fc model
class Classifier(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.layer1 = nn.Linear(784, 32)
        self.sigmoid1 = nn.Sigmoid()
        # self.relu1 = nn.ReLU()
        self.layer2 = nn.Linear(32, 10)
        self.sigmoid2 = nn.Sigmoid()
        # self.relu2 = nn.ReLU()
        pass

    def forward(self, input):
        output1 = self.layer1(input)
        output1 = self.sigmoid1(output1)
        output2 = self.layer2(output1)
        output2 = self.sigmoid2(output2)
        return output2

    def train(self, input):
        predicted = self.forward(input)
        return predicted

In [10]:
mnist_dataset = MnistDataset("data/mnist_test.csv")
classifier = Classifier()
loss_function = nn.MSELoss()
optimiser = optim.SGD(classifier.parameter(), learning_rate=0.1)

In [ ]:
image_label, image_content, target = mnist_dataset[2]
parameters_dict = classifier.state_dict()
for parameter_name in parameters_dict:
    print(parameter_name)

In [13]:
epochs = 1
count = 0
for epoch in range(epochs):
    print(f"epoch: {epoch + 1}")
    for image_label, image_data, image_target in mnist_dataset:
        pred = classifier.train(image_data.reshape([-1, 784]))
        count += 1
        loss = loss_function(pred, image_target.reshape([-1, 10]))
        # loss = (pred - image_target.reshape([-1, 10])) ** 2
        # loss.data /= 60000
        if count % 1000 == 0:
            print(loss.data)
            print(classifier.state_dict()["layer2.bias"].grad.data)
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()
    pass

epoch: 1
0.899949476287274
[ 1.38692488e-04  1.05976896e-04  1.37867992e-04  1.40903721e-04
  1.37937147e-04  1.29449833e-04  1.35547387e-04  1.34584951e-04
 -9.78144406e-09  1.37510537e-04]
0.8998811434316227
[ 1.24084674e-04  1.01658411e-04  1.24917695e-04  1.13361800e-04
  1.20582493e-04  1.26263906e-04  1.24796393e-04  1.28830034e-04
  1.25612213e-04 -6.00084802e-09]
0.8998823811330773
[ 2.31486462e-05  4.99482329e-05  1.29359785e-04  3.45638064e-05
 -1.38817247e-10  3.61887967e-05  8.86257907e-05  3.51029576e-05
  2.34842551e-05  7.22272229e-05]
0.8998968840262404
[ 7.45309035e-05  7.64842466e-05  5.54940171e-05  5.29272223e-05
 -2.98981470e-09  8.35897715e-05  8.62156688e-05  8.16649048e-05
  7.59318030e-05  7.87443393e-05]
0.899887288550951
[ 1.29685103e-04  1.02137123e-04  1.29939025e-04  1.33196749e-04
 -8.71770545e-09  1.21264270e-04  1.27878334e-04  1.27407384e-04
  1.31488399e-04  1.29785748e-04]
0.8999858131276722
[ 9.38716356e-06 -4.25464670e-09  1.81900159e-06  2.2102768